In [1]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth
import rasterio
from shapely.geometry import GeometryCollection, Polygon, box, shape, mapping
from google.cloud import storage

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
os.environ["GCS_RESOURCE_BUCKET"] = 'tomorrownow-actinia-dev'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './env/gcp_tomorrownow_dev_key.json'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

In [27]:
# !pip3 install google-cloud-storage 
!pip install rasterio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 17.3 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
--- Logging error ---
Traceback (most recent call last):
  File "/home/coreywhite/.local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/coreywhite/.local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/coreywhite/.local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/coreywhite/.local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/coreywhite/.local/lib/python3.10/site-p

In [2]:
from datetime import datetime

import pystac
print(pystac.__version__)

from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.view import ViewExtension

1.6.1


# Create a New root STAC catalog

## Collections

### geomorphology
#### elevation
- LandFire_CONUS
#### slope
- LandFirePercent_CONUS
- LandFireSlopeDegree_CONUS
#### aspect
- LandFire_CONUS
         
### hydrology
#### FlowDirection
- MeritHydro
#### Basins
- MeritHydro
#### Streams
- MeritHydro
#### WaterBodies

### transportation
#### Roads
- LandFire_CONUS
      
### lulc
#### Land Cover
- NLCD (2001 - 2019)
- [X] GCP
- [ ] Collection
#### Land Use
- NWALT (1974 - 2012)
#### Impervious Surface
- NLCD (2001 - 2019)
#### Protected Areas
- PADUS3_CONUS
         
 
### climate_weather
#### Precipitation
#### Climate Projections

### demographic
#### Population
- LandScanGlobal
    - Day (2000 - 2021)
    - Night (2000 - 2021)
- LandScanUSA 
     - Day (2016 - 2021)
     - Night (2016 - 2021)
     
### natural_disasters
#### Flooding
- FloodDamageProbability
#### Fire
     

Creating a root catalog to oragnize all data under

In [142]:
op_catalog = pystac.Catalog(id = "openplains", description= "STAC collection containing data used by OpenPlains.")

In [143]:
op_catalog.links

[<Link rel=root target=<Catalog id=openplains>>]

In [144]:
# Placeholder spatial extent will set from items once they are loaded
sp_extent = pystac.SpatialExtent([None,None,None,None])

## Add LULC catalog

Adding a child catalog to op_catalog to store land cover land use collections

In [145]:
lclu = pystac.Catalog(id = "lclu", description="A catalog containing land cover land use data collections")
op_catalog.add_child(lclu)

In [146]:
op_catalog.links

[<Link rel=root target=<Catalog id=openplains>>,
 <Link rel=child target=<Catalog id=lclu>>]

In [147]:
op_catalog.describe()

* <Catalog id=openplains>
    * <Catalog id=lclu>


### Add a collection to the lclu catalog

In [148]:
# bounding_box = (west, south, east, north)
# Upper Left  (, ) (128d23'12.86"W, 48d45' 5.48"N)
# Lower Left  (,  ) (118d41' 8.49"W, 22d25'42.26"N)
# Upper Right ( 2327655.000, ) ( 64d 3'14.58"W, 48d51'23.26"N)
# Lower Right ( 2327655.000) ( 73d38' 8.32"W, 22d30'13.03"N)

north = 3267405
west = -2362395
south = 221265
east = 2327655
sp_extent = pystac.SpatialExtent([None,None,None,None])
start_date = datetime.strptime('2001-01-01', '%Y-%m-%d')
end_data = datetime.strptime('2019-01-01', '%Y-%m-%d')
temporal_extent = pystac.TemporalExtent([(start_date, end_data)])
extent = pystac.Extent(sp_extent, temporal_extent)

In [149]:
nlcd = pystac.Collection(id='nlcd', description = 'NLCD datasets 2001 - 2019', extent = extent)
lclu.add_child(nlcd)

In [150]:
op_catalog.describe()

* <Catalog id=openplains>
    * <Catalog id=lclu>
        * <Collection id=nlcd>


### Load data from GCS into collection

In [37]:
from google.cloud import storage

In [39]:
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "tomorrownow-actinia-dev"
buckets = storage_client.list_buckets()

In [40]:
blobs = storage_client.list_blobs(bucket_name, prefix="nlcd")
data = [b.name for b in blobs if b.name.endswith('tif')]
print(data)

['nlcd/nlcd_2001_cog.tif', 'nlcd/nlcd_2004_cog.tif', 'nlcd/nlcd_2006_cog.tif', 'nlcd/nlcd_2008_cog.tif', 'nlcd/nlcd_2011_cog.tif', 'nlcd/nlcd_2013_cog.tif', 'nlcd/nlcd_2016_cog.tif', 'nlcd/nlcd_2019_cog.tif']


In [60]:
blobs = storage_client.list_blobs(bucket_name, prefix="nlcd")

for blob in blobs:
    if blob.name.endswith('tif'):
        print("Content-type: {}".format(blob.content_type))
        blob.content_type = "image/tiff; application=geotiff; profile=cloud-optimized"
        blob.patch()
        blob.make_public()

Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff
Content-type: image/tiff; application=geotiff; profile=cloud-optimized


#### Add items to collection

In [77]:
for d in data:
    uri = os.path.join(f'gs://{bucket_name}', d)
    uri = os.path.join(f'gs://{bucket_name}', d) nlcd_2001_cog.tif
    params = {}
    params['id'] = os.path.basename(uri).split('.')[0]
    with rasterio.open(uri) as src:
        params['bbox'] = list(src.bounds)
        params['geometry'] = mapping(box(*params['bbox']))
        
    params['datetime'] = datetime.strptime(f'{d.split("_")[1]}-01-01', '%Y-%m-%d')
    params['properties'] = {}
    i = pystac.Item(**params)
    i.add_asset(key='image', asset=pystac.Asset(href=uri,
                                                title='Geotiff',
                                                media_type=pystac.MediaType.COG))
    nlcd.add_item(i)

In [78]:
# lclu.remove_child('nlcd')
op_catalog.describe()

* <Catalog id=savana>
    * <Catalog id=lclu>
        * <Collection id=nlcd>
          * <Item id=nlcd_2001_cog>
          * <Item id=nlcd_2004_cog>
          * <Item id=nlcd_2006_cog>
          * <Item id=nlcd_2008_cog>
          * <Item id=nlcd_2011_cog>
          * <Item id=nlcd_2013_cog>
          * <Item id=nlcd_2016_cog>
          * <Item id=nlcd_2019_cog>


#### Update item extents

In [80]:
nlcd.update_extent_from_items()
nlcd.to_dict()

{'type': 'Collection',
 'id': 'nlcd',
 'stac_version': '1.0.0',
 'description': 'NLCD datasets 2001 - 2019',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': None,
   'type': <MediaType.J

## Add transportation Catalog

In [54]:
transportation = pystac.Catalog(id = "transportation", description="A catalog containing transportation data collections")
op_catalog.add_child(transportation)

### Add Collections

In [55]:
sp_extent = pystac.SpatialExtent([None,None,None,None])
roads = pystac.Collection(id='roads', description = 'Road Datasets', extent = sp_extent)
transportation.add_child(roads)

### Add items

In [56]:
def addCogAssets(uri, collection, properties={}, extra_asset_fields={}):
    with rasterio.open(uri) as src:
        params = {}
        params['id'] = os.path.basename(uri).split('.')[0]
    # try:
        params['bbox'] = list(src.bounds)
        params['geometry'] = mapping(box(*params['bbox']))

        params['datetime'] = datetime.strptime(f'{2020}-01-01', '%Y-%m-%d')
        params['properties'] = properties
        params["stac_extensions"]: [
            "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
            "https://stac-extensions.github.io/eo/v1.0.0/schema.json",
            "https://stac-extensions.github.io/classification/v1.1.0/schema.json"
        ]
        i = pystac.Item(**params)
        i.add_asset(key='analytic', asset=pystac.Asset(href=uri,
                                                    title='LF 2020 Operational Roads',
                                                    media_type=pystac.MediaType.COG,
                                                    roles=['data'], extra_fields=extra_asset_fields))
        collection.add_item(i)
    # except:
    #     print(e)
        
    
url = 'https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/transportation/roads/LC20_Roads_220_cog.tif'
properties = {
    "source": "https://www.landfire.gov/transportation.php",
    "name": "LF 2020 Operational Roads",
    "proj:epsg": 5070   
}

extra_asset_field = { 
    "raster:bands": [
        {
            "classification:classes": [
                {
                    "value": -9999,
                    "name": "Fill - NoData",
                    "description": "NoData",
                    "nodata": True
                },
                {
                    "value": 0,
                    "name": "Background Value",
                    "description": "Background Value.",
                    "color_hint": "000000"
                },
                {
                    "value": 20,
                    "name": "Primary road",
                    "description": "Interstates and other major roads. Pixels were derived from the 2018 NavStreets Street Data.",
                    "color_hint": "ff0101"
                },
                {
                    "value": 21,
                    "name": "Secondary road",
                    "description": "Non-interstate highways. Pixels were derived from the 2018 NavStreets Street Data.",
                    "color_hint": "ffff01"
                },
                {
                    "value": 22,
                    "name": "Tertiary road",
                    "description": "Any two-lane road. Pixels were derived from the 2018 NavStreets Street Data.",
                    "color_hint": "00f00f"
                },
                {
                    "value": 23,
                    "name": "Thinned road",
                    "description": "Small tertiary roads that generally are not paved and have been removed from the landcover but remain as part of the impervious surface product. Pixels were derived from the 2018 NavStreets Street Data",
                    "color_hint": "ffffff"
                },
                {
                    "value": 255,
                    "name": "NoData",
                    "description": "NoData",
                    "nodata": True
                }
          ]
        }
      ]
}
addCogAssets(url, roads, properties, extra_asset_field)

In [57]:
# op_catalog.remove_child('transportation')
op_catalog.describe()

* <Catalog id=openplains>
    * <Catalog id=lclu>
        * <Collection id=nlcd>
    * <Catalog id=transportation>
        * <Collection id=roads>
          * <Item id=LC20_Roads_220_cog>


In [58]:
roads.update_extent_from_items()
roads.to_dict()

{'type': 'Collection',
 'id': 'roads',
 'stac_version': '1.0.0',
 'description': 'Road Datasets',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': '/home/coreywhite/Documents/GitHub/TomorrowNow/TomorrowNowApp/notebooks/op_catalog/catalog.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.SELF: 'self'>,
   'href': '/home/coreywhite/Documents/GitHub/TomorrowNow/TomorrowNowApp/notebooks/op_catalog/transportation/roads/collection.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ITEM: 'item'>,
   'href': '/home/coreywhite/Documents/GitHub/TomorrowNow/TomorrowNowApp/notebooks/op_catalog/transportation/roads/LC20_Roads_220_cog/LC20_Roads_220_cog.json',
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.PARENT: 'parent'>,
   'href': '/home/coreywhite/Documents/GitHub/TomorrowNow/TomorrowNowApp/notebooks/op_catalog/transportation/catalog.json',
   'type': <MediaType.JSON: 'application/json'>}],
 'stac_extensions': [],
 '

## Add Geomorphology Catalog

## Normalize Catalogs hrefs

In [88]:
# op_catalog.normalize_hrefs(os.path.join(tmp_dir.name, 'stac'))
op_catalog.normalize_hrefs('./op_catalog')
op_catalog.save("./op_catalog")

# GRASSDATA Catalog

In [134]:
grassdata_catalog = pystac.Catalog(id = "grassdata", description= "GRASS GIS STAC data catalog used by OpenPlains.")

sp_extent = pystac.SpatialExtent([33.83,-84.33,36.59,-75.38])
# A GRASS Location
nc_spm_08 = pystac.Collection(
    id="nc_spm_08",
    description="GRASS GIS Sample Datasets",
    extent=sp_extent,
    title="nc_spm_08",
    stac_extensions=[
        "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
        "https://stac-extensions.github.io/scientific/v1.0.0/schema.json"
    ],
    license="GNU General Public License (GPL)",
    keywords=['GRASS GIS', 'Location'],
    extra_fields={
        "grass:type": "location",
        "proj:epsg": 3358,
        "sci:citation": "GRASS Development Team, 2022. Geographic Resources Analysis Support System (GRASS) Software, Version 8.0. Open Source Geospatial Foundation. https://grass.osgeo.org" 
    }
)
grassdata_catalog.add_child(nc_spm_08)

In [108]:
grassdata_catalog.to_dict()
nc_spm_08.to_dict()

{'type': 'Collection',
 'id': 'nc_spm_08',
 'stac_version': '1.0.0',
 'description': 'GRASS GIS Sample Datasets',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.PARENT: 'parent'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>}],
 'stac_extensions': ['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
  'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
 'grass:type': 'location',
 'proj:epsg': 3358,
 'sci:citation': 'GRASS Development Team, 2022. Geographic Resources Analysis Support System (GRASS) Software, Version 8.0. Open Source Geospatial Foundation. https://grass.osgeo.org',
 'title': 'nc_spm_08',
 'extent': {'bbox': [[33.83, -84.33, 36.59, -75.38]]},
 'license': 'GNU General Public License (GPL)',
 'keywords': ['GRASS GIS', 'Location']}

In [135]:
PERMANENT = pystac.Collection(
    id="PERMANENT",
    description="defualt mapset",
    extent=sp_extent,
    title="PERMANENT",
    stac_extensions=[
        "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
        "https://stac-extensions.github.io/scientific/v1.0.0/schema.json"
    ],
    license="GNU General Public License (GPL)",
    keywords=['GRASS GIS', 'mapset', 'PERMANENT'],
    extra_fields={
        "grass:type": "mapset",
        "proj:epsg": 3358,
        "sci:citation": "GRASS Development Team, 2022. Geographic Resources Analysis Support System (GRASS) Software, Version 8.0. Open Source Geospatial Foundation. https://grass.osgeo.org" 
    }
)
nc_spm_08.add_child(PERMANENT)

In [113]:
PERMANENT.to_dict()

{'type': 'Collection',
 'id': 'PERMANENT',
 'stac_version': '1.0.0',
 'description': 'defualt mapset',
 'links': [{'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.PARENT: 'parent'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>,
   'title': 'nc_spm_08'}],
 'stac_extensions': ['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
  'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
 'grass:type': 'mapset',
 'proj:epsg': 3358,
 'sci:citation': 'GRASS Development Team, 2022. Geographic Resources Analysis Support System (GRASS) Software, Version 8.0. Open Source Geospatial Foundation. https://grass.osgeo.org',
 'title': 'PERMANENT',
 'extent': {'bbox': [[33.83, -84.33, 36.59, -75.38]]},
 'license': 'GNU General Public License (GPL)',
 'keywords': ['GRASS GIS', 'mapset', 'PERMANENT']}

In [99]:
# elevation_json = !curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation" | jq

url =  "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation"
# # url = "http://localhost:8088/api/v3/locations/the_research_triangle/process_chain_validation_sync"
r = requests.get(
            url, 
            auth=ACTINIA_AUTH,
            headers={"content-type": "application/json; charset=utf-8"}
        )
elevation_json = r.json()
elevation_json
# print(f"Response: {r.json()}")

{'accept_datetime': '2022-11-11 07:34:18.163788',
 'accept_timestamp': 1668152058.1637862,
 'api_info': {'endpoint': 'rasterlayerresource',
  'method': 'GET',
  'path': '/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation',
  'request_url': 'http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation'},
 'datetime': '2022-11-11 07:34:18.362907',
 'http_code': 200,
 'message': 'Processing successfully finished',
 'process_chain_list': [{'1': {'flags': 'gre',
    'inputs': {'map': 'elevation@PERMANENT'},
    'module': 'r.info'}}],
 'process_log': [{'executable': 'r.info',
   'id': '1',
   'mapset_size': 421,
   'parameter': ['map=elevation@PERMANENT', '-gre'],
   'return_code': 0,
   'run_time': 0.10041546821594238,
   'stderr': [''],
   'stdout': 'north=228500\nsouth=215000\neast=645000\nwest=630000\nnsres=10\newres=10\nrows=1350\ncols=1500\ncells=2025000\ndatatype=FCELL\nncats=255\nmin=55.57879\nmax=156.3299\nmap=elevation\nmaptype=raste

In [136]:
def addGrassAssets(collection, location="nc_spm_08", mapset="PERMANENT", data_type="raster_layers", data_name="elevation"):
    url =  f"http://localhost:8088/api/v3/locations/{location}/mapsets/{mapset}/{data_type}/{data_name}"
    r = requests.get(
                url, 
                auth=ACTINIA_AUTH,
                headers={"content-type": "application/json; charset=utf-8"}
            )
    response_json = r.json()
    if response_json:
        uri = response_json['api_info']['path']
        params = {}
        params['id'] = data_name
        process_results = response_json['process_results']
        # left, bottom, right, top
        params['bbox'] = [int(x) for x in [process_results['west'],process_results['south'],process_results['east'],process_results['north']]]
        params['geometry'] = mapping(box(*params['bbox']))

        params['datetime'] = datetime.strptime("2006-11-07T01:09:51", '%Y-%d-%mT%H:%M:%S') #response_json['accept_datetime'] #process_results['date']
        
        # Some of these may need to be placed into other extensions (e.g., raster, proj, etc..)
        grass_extensions = ['datatype', 'comments', 'creator', 
                          'ewres', 'nsres', 'cols', 'location', 'mapset', 'map', 'maptype', 'min', 'max', 'ncats', 'semantic_label', 'source1', 'source2']
        
        params['properties'] = {
            "title": process_results['title'],
            "description": process_results['description'],
            "proj:epsg": 3358
        }
        
        for ge in grass_extensions:
            label = f"grass:{ge}"
            params['properties'][label] = process_results[ge]
            
        params["stac_extensions"]: [
            "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
            # "https://stac-extensions.github.io/eo/v1.0.0/schema.json",
            # "https://stac-extensions.github.io/classification/v1.1.0/schema.json",
            # "https://stac-extensions.github.io/raster/v1.1.0/schema.json"
        ]
        i = pystac.Item(**params)
        
        ## Add GRASS Colors
        extra_asset_fields={}
        
        i.add_asset(key=process_results['maptype'], asset=pystac.Asset(href=uri,
                                                    title=process_results['title'],
                                                    media_type=pystac.MediaType.COG,
                                                    roles=['data'], extra_fields=extra_asset_fields))
        
        i.add_asset(key='thumbnail', asset=pystac.Asset(href=uri + 'render',
                                                    title=process_results['title'] + " " + "Thumbnail",
                                                    media_type=pystac.MediaType.PNG,
                                                    roles=['thumbnail']))
        
        
        collection.add_item(i)
        
    


addGrassAssets(PERMANENT)

In [137]:
grassdata_catalog.describe()
PERMANENT.get_item("elevation")

* <Catalog id=grassdata>
    * <Collection id=nc_spm_08>
        * <Collection id=PERMANENT>
          * <Item id=elevation>


ID: elevation
"Bounding Box: [630000, 215000, 645000, 228500]"
Datetime: 2006-07-11 01:09:51
"title: ""South-West Wake county: Elevation NED 10m"""
"description: ""generated by r.proj"""
proj:epsg: 3358
grass:datatype: FCELL
"grass:comments: ""r.proj input=""ned03arcsec"" location=""northcarolina_latlong"" mapset=""\helena"" output=""elev_ned10m"" method=""cubic"" resolution=10"""
"grass:creator: ""helena"""
grass:ewres: 10
grass:nsres: 10


In [140]:
grassdata_catalog.normalize_hrefs('https://github.com/tomorrownow/TomorrowNowApp/blob/main/notebooks/grass_catalog/')

In [141]:
grassdata_catalog.save("./grass_catalog")